# Imagery Insights eval

> Add blockquote



This notebook demonstrates how to classify images from GCS URIs using the Gemini 2.5 Flash model via Google Cloud's Vertex AI.

## Install Required Libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform pandas

## Imports and Vertex AI Initialization

In [97]:
import vertexai
from google.cloud import bigquery
from google.cloud import storage
from google import genai
from google.genai.types import Content, Part
import pandas as pd
from IPython.display import display
from collections import Counter, defaultdict
import os, re, base64
import json

In [108]:
# --- Configuration ---
PROJECT_ID = 'sarthaks-lab'  # @param {type:"string"}
REGION = 'us-central1'      # @param {type:"string"}
DATASET_ID = 'imagery_insights___preview___us' # @param {type:"string"}
TABLE_ID = 'latest_observations' # @param {type:"string"}
QUERY_LIMIT = 10 # @param {type:"integer"}
MODEL_FLASH = 'gemini-2.5-flash' # @param {type:"string"}
MODEL_PRO = 'gemini-2.5-pro' # @param {type:"string"}

## Initialize Vertex AI SDK

In [99]:
# --- Initialize Vertex AI SDK ---
vertexai.init(project=PROJECT_ID, location=REGION)

print("✅ Setup Complete: Libraries installed, configured, and Vertex AI initialized.")

✅ Setup Complete: Libraries installed, configured, and Vertex AI initialized.


In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "61d1bfb1"
      },
      "source": [
        "# Imagery Insights eval\n",
        "\n",
        "> Add blockquote\n",
        "\n",
        "\n",
        "\n",
        "This notebook demonstrates how to classify images from GCS URIs using the Gemini 2.5 Flash model via Google Cloud's Vertex AI."
      ],
      "id": "61d1bfb1"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ab074d90"
      },
      "source": [
        "## Install Required Libraries"
      ],
      "id": "ab074d90"
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "23f28877"
      },
      "source": [
        "!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform pandas"
      ],
      "id": "23f28877",
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "64efff2f"
      },
      "source": [
        "## Imports and Vertex AI Initialization"
      ],
      "id": "64efff2f"
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "RPWtl7xgWjZB"
      },
      "source": [
        "import vertexai\n",
        "from google.cloud import bigquery\n",
        "from google.cloud import storage\n",
        "from google import genai\n",
        "from google.genai.types import Content, Part\n",
        "import pandas as pd\n",
        "from IPython.display import display\n",
        "from collections import Counter, defaultdict\n",
        "import os, re, base64\n",
        "import json"
      ],
      "execution_count": 97,
      "outputs": [],
      "id": "RPWtl7xgWjZB"
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "b0d35a79"
      },
      "source": [
        "# --- Configuration ---\n",
        "PROJECT_ID = ''  # @param {type:\"string\"}\n",
        "REGION = 'us-central1'      # @param {type:\"string\"}\n",
        "DATASET_ID = '' # @param {type:\"string\"}\n",
        "TABLE_ID = '' # @param {type:\"string\"}\n",
        "QUERY_LIMIT = 10 # @param {type:\"integer\"}\n",
        "MODEL_FLASH = 'gemini-2.5-flash' # @param {type:\"string\"}\n",
        "MODEL_PRO = 'gemini-2.5-pro' # @param {type:\"string\"}"
      ],
      "id": "b0d35a79",
      "execution_count": 108,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "## Initialize Vertex AI SDK"
      ],
      "metadata": {
        "id": "60YpCxxOXBY3"
      },
      "id": "60YpCxxOXBY3"
    },
    {
      "cell_type": "code",
      "source": [
        "# --- Initialize Vertex AI SDK ---\n",
        "vertexai.init(project=PROJECT_ID, location=REGION)\n",
        "\n",
        "print(\"✅ Setup Complete: Libraries installed, configured, and Vertex AI initialized.\")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "9jIiydZhXG9H",
        "outputId": "58b61ff1-f211-4fe9-aa3c-c13319b8c1cb"
      },
      "id": "9jIiydZhXG9H",
      "execution_count": 99,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "✅ Setup Complete: Libraries installed, configured, and Vertex AI initialized.\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "## Create vertex client"
      ],
      "metadata": {
        "id": "AVLn1AU4X4pJ"
      },
      "id": "AVLn1AU4X4pJ",
      "execution_count": 100,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)\n"
      ],
      "metadata": {
        "id": "p9f_p4mTXvHl"
      },
      "id": "p9f_p4mTXvHl",
      "execution_count": 101,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "##Compares two model outputs to see if they are semantically the same.\n",
        "\n",
        "    Args:\n",
        "        output1: The first model's JSON output.\n",
        "        output2: The second model's JSON output.\n",
        "        model_name: The name of the Gemini model to use for comparison.\n",
        "\n",
        "    Returns:\n",
        "        'Yes' if the outputs are semantically the same, 'No' otherwise."
      ],
      "metadata": {
        "id": "H0CmOUM3XhJQ"
      },
      "id": "H0CmOUM3XhJQ"
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "e2fc17ff"
      },
      "source": [
        "def judge_outputs(output1: str, output2: str) -> str:\n",
        "\n",
        "    comparison_prompt = f\"\"\"You will be provided with two JSON objects. Your task is to determine if they are semantically the same.\n",
        "\n",
        "    Respond with 'SAME' if they are semantically the same, and 'DIFFERENT' if they are not.\n",
        "\n",
        "    Add key points that you saw as different.\n",
        "\n",
        "    JSON 1:\n",
        "    {output1}\n",
        "\n",
        "    JSON 2:\n",
        "    {output2}\n",
        "    \"\"\"\n",
        "    contents = [\n",
        "    comparison_prompt]\n",
        "\n",
        "    try:\n",
        "\n",
        "        response = client.models.generate_content(model=MODEL_FLASH, contents=contents)\n",
        "\n",
        "        return response.text\n",
        "    except Exception as e:\n",
        "        print(f\"Error during comparison: {e}\")\n",
        "        return \"Comparison failed.\""
      ],
      "id": "e2fc17ff",
      "execution_count": 102,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "## Classify image with Gemini"
      ],
      "metadata": {
        "id": "ZQ7-wK-7Z4HU"
      },
      "id": "ZQ7-wK-7Z4HU"
    },
    {
      "cell_type": "code",
      "source": [
        "def classify_image_with_gemini(gcs_uri: str, prompt: str, model_name: str) -> str:\n",
        "    \"\"\"Classifies an image using a Gemini model.\"\"\"\n",
        "\n",
        "    # The 'mime_type' must be passed as a keyword argument.\n",
        "    contents = [\n",
        "    prompt,\n",
        "    Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]\n",
        "\n",
        "    response = client.models.generate_content(model=model_name, contents=contents)\n",
        "    return response.text"
      ],
      "metadata": {
        "id": "ZOqqHCq0ZLL1"
      },
      "id": "ZOqqHCq0ZLL1",
      "execution_count": 106,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "## Helper function for json sanitization"
      ],
      "metadata": {
        "id": "4ANvDJh8fmLA"
      },
      "id": "4ANvDJh8fmLA"
    },
    {
      "cell_type": "code",
      "source": [
        "def _extract_json(text: str) -> str:\n",
        "    \"\"\"Extracts the JSON string from a text block, even if it's wrapped in markdown.\"\"\"\n",
        "    # Use a regex to find the content between ```json and ```\n",
        "    match = re.search(r\"```json\\s*(\\{.*?\\})\\s*```\", text, re.DOTALL)\n",
        "    if match:\n",
        "        return match.group(1)\n",
        "    # Fallback for just ```\n",
        "    match = re.search(r\"```\\s*(\\{.*?\\})\\s*```\", text, re.DOTALL)\n",
        "    if match:\n",
        "        return match.group(1)\n",
        "    # If no markdown, assume the whole string is the json\n",
        "    return text"
      ],
      "metadata": {
        "id": "ziWrCsbIfNdm"
      },
      "id": "ziWrCsbIfNdm",
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "## Prompt to extract information from given images"
      ],
      "metadata": {
        "id": "9ng81ueBfruM"
      },
      "id": "9ng81ueBfruM"
    },
    {
      "cell_type": "code",
      "source": [
        "prompt = \"\"\"You will be provided with a photo of a utility pole.\n",
        "User Input:\n",
        "{\n",
        "photo_of_utility_pole\n",
        "}\n",
        "\n",
        "Follow these instructions to analyze the image:\n",
        "\n",
        "1.  **Analyze the Image:** Carefully examine the provided image. If the image does not clearly show a utility pole, you must return the following JSON object and stop processing:\n",
        "    `{\\\"error\\\": \\\"No utility pole detected in the image.\\\"}`\n",
        "\n",
        "2.  **Identify Key Information:** If a utility pole is present, identify the following information from the image:\n",
        "    *   **Material:** Determine the material the pole is made from (e.g., \\\"wood\\\", \\\"metal\\\", \\\"concrete\\\").\n",
        "    *   **Type:** Determine the primary type of the pole. Choose from: \\\"Street light\\\", \\\"High tension power transmission\\\", \\\"electricity pole\\\", or \\\"other\\\".\n",
        "\n",
        "\n",
        "3.  **Format the Output:** Your response must be the raw JSON object only, without any surrounding text, explanations, or markdown formatting like ```json.\n",
        "\n",
        "**Output Format:**\n",
        "{\n",
        "  \\\"material\\\": \\\"[material]\\\",\n",
        "  \\\"type\\\": \\\"[pole_type]\\\"\n",
        "\n",
        "}\n",
        "\n",
        "**Example:**\n",
        "\n",
        "If the provided image shows a wooden electricity pole with a transformer, power lines, and a street lamp, your output should be:\n",
        "\n",
        "{\n",
        "  \\\"material\\\": \\\"wood\\\",\n",
        "  \\\"type\\\": \\\"electricity pole\\\"\n",
        "\n",
        "}\n",
        "\"\"\""
      ],
      "metadata": {
        "id": "fcpsjYgafYD6"
      },
      "id": "fcpsjYgafYD6",
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "## Main processing block"
      ],
      "metadata": {
        "id": "fBZu_G4Ejr06"
      },
      "id": "fBZu_G4Ejr06"
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "732877cf"
      },
      "source": [
        "results_list = []\n",
        "\n",
        "try:\n",
        "    # Fetch data from BigQuery\n",
        "    BIGQUERY_SQL_QUERY = f\"\"\"\n",
        "    SELECT\n",
        "      gcs_uri\n",
        "    FROM\n",
        "      `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`\n",
        "      WHERE asset_type = \"ASSET_CLASS_UTILITY_POLE\"\n",
        "\n",
        "    LIMIT {QUERY_LIMIT}\n",
        "    \"\"\"\n",
        "    bigquery_client = bigquery.Client(project=PROJECT_ID)\n",
        "    eval_dataset = bigquery_client.query(BIGQUERY_SQL_QUERY).to_dataframe()\n",
        "    print(f\"Successfully fetched {len(eval_dataset)} records for evaluation.\")\n",
        "\n",
        "    # Loop through the dataset\n",
        "    for index, row in eval_dataset.iterrows():\n",
        "        uri = row['gcs_uri']\n",
        "        print(f\"--- ({index + 1}/{len(eval_dataset)}) Evaluating {uri} ---\")\n",
        "\n",
        "        flash_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_FLASH)\n",
        "        pro_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_PRO)\n",
        "\n",
        "        # Extract clean JSON strings\n",
        "        flash_classification_str = _extract_json(flash_classification_raw)\n",
        "        pro_classification_str = _extract_json(pro_classification_raw)\n",
        "\n",
        "        # Parse into JSON objects with error handling\n",
        "        try:\n",
        "            flash_output_json = json.loads(flash_classification_str)\n",
        "        except json.JSONDecodeError:\n",
        "            flash_output_json = {'error': 'Failed to parse JSON from flash model.', 'raw_output': flash_classification_raw}\n",
        "\n",
        "        try:\n",
        "            pro_output_json = json.loads(pro_classification_str)\n",
        "        except json.JSONDecodeError:\n",
        "            pro_output_json = {'error': 'Failed to parse JSON from pro model.', 'raw_output': pro_classification_raw}\n",
        "\n",
        "        judgement = judge_outputs(json.dumps(flash_output_json), json.dumps(pro_output_json))\n",
        "\n",
        "        results_list.append({\n",
        "            'gcs_uri': uri,\n",
        "            'flash_output': flash_output_json,\n",
        "            'pro_output': pro_output_json,\n",
        "            'judgement': judgement\n",
        "        })\n",
        "        print(f\"Verdict: {judgement}\")\n",
        "\n",
        "    results_df = pd.DataFrame(results_list)\n",
        "    print(\"\\n✅ Evaluation Complete. Analyzing results...\")\n",
        "\n",
        "    agreement_count = results_df['judgement'].str.startswith('SAME').sum()\n",
        "    total_count = len(results_df)\n",
        "    agreement_percentage = (agreement_count / total_count) * 100 if total_count > 0 else 0\n",
        "    print(f\"\\n--- Aggregate Results ---\")\n",
        "    print(f\"Overall Model Agreement: {agreement_percentage:.2f}%\\n\")\n",
        "\n",
        "    disagreements_df = results_df[results_df['judgement'].str.startswith('DIFFERENT')].copy()\n",
        "    print(\"--- Examples of Disagreements ---\")\n",
        "\n",
        "    if not disagreements_df.empty:\n",
        "        pd.set_option('display.max_colwidth', None)\n",
        "        display(disagreements_df.head(3))\n",
        "    else:\n",
        "        print(\"No disagreements found.\")\n",
        "\n",
        "except Exception as e:\n",
        "    print(f\"An error occurred during the evaluation pipeline: {e}\")"
      ],
      "id": "732877cf",
      "execution_count": null,
      "outputs": []
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.10.10"
    },
    "colab": {
      "provenance": [],
      "name": "utility_pole_basic_analysis_eval.ipynb"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

In [101]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)


##Compares two model outputs to see if they are semantically the same.

    Args:
        output1: The first model's JSON output.
        output2: The second model's JSON output.
        model_name: The name of the Gemini model to use for comparison.

    Returns:
        'Yes' if the outputs are semantically the same, 'No' otherwise.

In [102]:
def judge_outputs(output1: str, output2: str) -> str:

    comparison_prompt = f"""You will be provided with two JSON objects. Your task is to determine if they are semantically the same.

    Respond with 'SAME' if they are semantically the same, and 'DIFFERENT' if they are not.

    Add key points that you saw as different.

    JSON 1:
    {output1}

    JSON 2:
    {output2}
    """
    contents = [
    comparison_prompt]

    try:

        response = client.models.generate_content(model=MODEL_FLASH, contents=contents)

        return response.text
    except Exception as e:
        print(f"Error during comparison: {e}")
        return "Comparison failed."

## Classify image with Gemini

In [106]:
def classify_image_with_gemini(gcs_uri: str, prompt: str, model_name: str) -> str:
    """Classifies an image using a Gemini model."""

    # The 'mime_type' must be passed as a keyword argument.
    contents = [
    prompt,
    Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

    response = client.models.generate_content(model=model_name, contents=contents)
    return response.text

## Helper function for json sanitization

In [ ]:
def _extract_json(text: str) -> str:
    """Extracts the JSON string from a text block, even if it's wrapped in markdown."""
    # Use a regex to find the content between ```json and ```
    match = re.search(r"```json\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    # Fallback for just ```
    match = re.search(r"```\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    # If no markdown, assume the whole string is the json
    return text

## Prompt to extract information from given images

In [ ]:
prompt = """You will be provided with a photo of a utility pole.
User Input:
{
photo_of_utility_pole
}

Follow these instructions to analyze the image:

1.  **Analyze the Image:** Carefully examine the provided image. If the image does not clearly show a utility pole, you must return the following JSON object and stop processing:
    `{\"error\": \"No utility pole detected in the image.\"}`

2.  **Identify Key Information:** If a utility pole is present, identify the following information from the image:
    *   **Material:** Determine the material the pole is made from (e.g., \"wood\", \"metal\", \"concrete\").
    *   **Type:** Determine the primary type of the pole. Choose from: \"Street light\", \"High tension power transmission\", \"electricity pole\", or \"other\".


3.  **Format the Output:** Your response must be the raw JSON object only, without any surrounding text, explanations, or markdown formatting like ```json.

**Output Format:**
{
  \"material\": \"[material]\",
  \"type\": \"[pole_type]\"

}

**Example:**

If the provided image shows a wooden electricity pole with a transformer, power lines, and a street lamp, your output should be:

{
  \"material\": \"wood\",
  \"type\": \"electricity pole\"

}
"""

## Main processing block

In [ ]:
results_list = []

try:
    # Fetch data from BigQuery
    BIGQUERY_SQL_QUERY = f"""
    SELECT
      gcs_uri
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
      WHERE asset_type = "ASSET_CLASS_UTILITY_POLE"

    LIMIT {QUERY_LIMIT}
    """
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    eval_dataset = bigquery_client.query(BIGQUERY_SQL_QUERY).to_dataframe()
    print(f"Successfully fetched {len(eval_dataset)} records for evaluation.")

    # Loop through the dataset
    for index, row in eval_dataset.iterrows():
        uri = row['gcs_uri']
        print(f"--- ({index + 1}/{len(eval_dataset)}) Evaluating {uri} ---")

        flash_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_FLASH)
        pro_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_PRO)

        # Extract clean JSON strings
        flash_classification_str = _extract_json(flash_classification_raw)
        pro_classification_str = _extract_json(pro_classification_raw)

        # Parse into JSON objects with error handling
        try:
            flash_output_json = json.loads(flash_classification_str)
        except json.JSONDecodeError:
            flash_output_json = {'error': 'Failed to parse JSON from flash model.', 'raw_output': flash_classification_raw}

        try:
            pro_output_json = json.loads(pro_classification_str)
        except json.JSONDecodeError:
            pro_output_json = {'error': 'Failed to parse JSON from pro model.', 'raw_output': pro_classification_raw}

        judgement = judge_outputs(json.dumps(flash_output_json), json.dumps(pro_output_json))

        results_list.append({
            'gcs_uri': uri,
            'flash_output': flash_output_json,
            'pro_output': pro_output_json,
            'judgement': judgement
        })
        print(f"Verdict: {judgement}")

    results_df = pd.DataFrame(results_list)
    print("\n✅ Evaluation Complete. Analyzing results...")

    agreement_count = results_df['judgement'].str.startswith('SAME').sum()
    total_count = len(results_df)
    agreement_percentage = (agreement_count / total_count) * 100 if total_count > 0 else 0
    print(f"\n--- Aggregate Results ---")
    print(f"Overall Model Agreement: {agreement_percentage:.2f}%\n")

    disagreements_df = results_df[results_df['judgement'].str.startswith('DIFFERENT')].copy()
    print("--- Examples of Disagreements ---")

    if not disagreements_df.empty:
        pd.set_option('display.max_colwidth', None)
        display(disagreements_df.head(3))
    else:
        print("No disagreements found.")

except Exception as e:
    print(f"An error occurred during the evaluation pipeline: {e}")